In [1]:
from numpy import save, load
from pandas import read_csv
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense,Activation
from keras.models import Input, Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,TimeDistributed, MaxPooling1D
import tensorflow as tf
import numpy as np
import keras.backend as K
import keras.layers
from keras import optimizers
from keras.layers import Activation, Lambda
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from typing import List, Tuple
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/MyDrive/Colab Notebooks/Dataset1/trainX.npy')
y_train = np.load('gdrive/MyDrive/Colab Notebooks/Dataset1/trainy.npy')
x_test = np.load('gdrive/MyDrive/Colab Notebooks/Dataset1/testX.npy')
y_test = np.load('gdrive/MyDrive/Colab Notebooks/Dataset1/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/gdrive
(33104, 128, 3) (33104, 118) (3740, 128, 3) (3740, 118)


In [3]:
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [4]:
def channel_normalization(x):
    max_values = K.max(K.abs(x), 2, keepdims=True) + 1e-5
    out = x / max_values
    return out

def residual_block(x, s, i, activation, nb_filters, kernel_size, padding, dropout_rate=0, name=''):
    original_x = x
    conv = Conv1D(filters=nb_filters, kernel_size=kernel_size,
                  dilation_rate=i, padding=padding,
                  name=name + '_dilated_conv_%d_tanh_s%d' % (i, s))(x)
    if activation == 'norm_relu':
        x = Activation('relu')(conv)
        x = Lambda(channel_normalization)(x)
    else:
        x = Activation(activation)(conv)

    x = SpatialDropout1D(dropout_rate, name=name + '_spatial_dropout1d_%d_s%d_%f' % (i, s, dropout_rate))(x)

    # 1x1 conv.
    x = Convolution1D(nb_filters, 1, padding='same')(x)
    res_x = keras.layers.add([original_x, x])
    return res_x, x

In [5]:
class TCN:

    def __init__(self,
                 nb_filters=64,
                 kernel_size=2,
                 nb_stacks=1,
                 dilations=None,
                 activation='norm_relu',
                 padding='causal',
                 use_skip_connections=True,
                 dropout_rate=0.0,
                 return_sequences=True,
                 name='tcn'):
        self.name = name
        self.return_sequences = return_sequences
        self.dropout_rate = dropout_rate
        self.use_skip_connections = use_skip_connections
        self.activation = activation
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters
        self.padding = padding
        
        if padding != 'causal' and padding != 'same':
            raise ValueError("Only 'causal' or 'same' paddings are compatible for this layer.")

        if not isinstance(nb_filters, int):
            print('An interface change occurred after the version 2.1.2.')
            print('Before: tcn.TCN(i, return_sequences=False, ...)')
            print('Now should be: tcn.TCN(return_sequences=False, ...)(i)')
            print('Second solution is to pip install keras-tcn==2.1.2 to downgrade.')
            raise Exception()

    def __call__(self, inputs):
        if self.dilations is None:
            self.dilations = [1, 2, 4, 8, 16, 32]
        x = inputs
        x = Convolution1D(self.nb_filters, 1, padding=self.padding, name=self.name + '_initial_conv')(x)
        skip_connections = []
        for s in range(self.nb_stacks):
            for i in self.dilations:
                x, skip_out = residual_block(x, s, i, self.activation, self.nb_filters,
                                             self.kernel_size, self.padding, self.dropout_rate, name=self.name)
                skip_connections.append(skip_out)
        if self.use_skip_connections:
            x = keras.layers.add(skip_connections)
        x = Activation('relu')(x)

        if not self.return_sequences:
            output_slice_index = -1
            x = Lambda(lambda tt: tt[:, output_slice_index, :])(x)
        return x

In [6]:
callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

In [7]:
input_shape = x_train.shape[1:]
input = Input(shape=x_train.shape[1:])

In [8]:
y_train.shape

(26483, 118)

In [9]:
y_train = (np.argmax(y_train, axis=1)).reshape(-1, 1)
y_test =  (np.argmax(y_test, axis=1)).reshape(-1, 1)

In [10]:
y_train.shape

(26483, 1)

In [12]:
acc_per_fold = []
loss_per_fold = []
num_folds = 2
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)
skf = StratifiedKFold(n_splits=5, shuffle=True)
fold_no = 1
input_shape = x_train.shape[1:]
input = Input(shape=x_train.shape[1:])
for train, test in skf.split(inputs, targets):

  x = SpatialDropout1D(0.2)(input)
  x = TCN(128,dilations = [1, 2, 4, 8, 16],kernel_size = 3, return_sequences=True, name = 'tnc1')(x)
  x = TCN(128,dilations = [1, 2, 4, 8, 16],kernel_size = 3, return_sequences=True, name = 'tnc2')(x)
  max_pool = GlobalMaxPooling1D()(x)
  x = Dense(128, activation="relu")(max_pool)
  x = Dropout(0.2)(x)
  output = Dense(118, activation="softmax")(x)    
  model = Model(inputs=input, outputs=output)
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  history = model.fit(inputs[train], targets[train], epochs=200, verbose=True, batch_size=128,callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=5,restore_best_weights=True)])
    # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

model.summary()

Epoch 1/200
189/189 [==============================] - 34s 163ms/step - loss: 3.1501 - accuracy: 0.2784
Epoch 2/200
189/189 [==============================] - 29s 151ms/step - loss: 1.4035 - accuracy: 0.6513
Epoch 3/200
189/189 [==============================] - 28s 151ms/step - loss: 0.9264 - accuracy: 0.7648
Epoch 4/200
189/189 [==============================] - 28s 150ms/step - loss: 0.7409 - accuracy: 0.8118
Epoch 5/200
189/189 [==============================] - 28s 150ms/step - loss: 0.6248 - accuracy: 0.8408
Epoch 6/200
189/189 [==============================] - 28s 150ms/step - loss: 0.5703 - accuracy: 0.8541
Epoch 7/200
189/189 [==============================] - 28s 150ms/step - loss: 0.5024 - accuracy: 0.8724
Epoch 8/200
189/189 [==============================] - 28s 150ms/step - loss: 0.4592 - accuracy: 0.8800
Epoch 9/200
189/189 [==============================] - 28s 149ms/step - loss: 0.4248 - accuracy: 0.8899
Epoch 10/200
189/189 [==============================] - 28s 150m

In [13]:
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.15169847011566162 - Accuracy: 96.24482989311218%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.11963732540607452 - Accuracy: 97.20430374145508%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.19478358328342438 - Accuracy: 95.07030844688416%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.1625102460384369 - Accuracy: 96.04566693305969%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.14939068257808685 - Accuracy: 95.94639539718628%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 96.10230088233948 (+- 0.6822385406527091)
> Loss: 0.15560406148433686
-----------------------